# Make crosswalk of small neighborhoods to aggregated neighborhoods with population 

In [2]:
import pandas as pd

In [4]:
df = pd.read_excel("../data/crosswalk_source.xlsx")

In [22]:
df = df.assign(
    Region = df.City_Community_Label.str.split("--", expand=True)[0],
)

# Split and then make long
make_long = (df.Included.str.split(";", expand=True)
             .assign(index = make_long.index)
            )

m1 = pd.melt(make_long, id_vars = "index", var_name = "col_name", value_name = "neighborhood")
m1 = m1[m1.neighborhood.notna()]

# Need merge var to be the same
df = df.assign(index = df.index)

# Merge in
df2 = pd.merge(df, m1, on = "index", how = "left", validate = "1:m")

df2 = (df2.drop(columns = ["City_Community_Label", "Included", "index", "col_name"])
        .assign(
            Region = df2.Region.str.replace("\n", " ")
        )
      )

In [46]:
df2['neighborhood'] = df2.neighborhood.str.replace("Unincorporated-", "Unincorporated - ")
df2['neighborhood'] = df2.neighborhood.str.replace("Los Angeles-", "Los Angeles - ")

In [47]:
df2['neighborhood'] = df2.neighborhood.str.strip()
df2['Region'] = df2.Region.str.strip()

In [50]:
df2 = df2.rename(columns = {"Region_Number":"region_num", "Combined_Population": "population",
                           "Region": "region"})

In [52]:
df2.to_csv("../data/la_county_neighborhoods_crosswalk.csv")
df2.to_parquet("../data/la_county_neighborhoods_crosswalk.parquet")